In [1]:
import numpy as np
# import pyvista as pv
from collections import defaultdict, deque
import copy

# ----------- Normal vectors estimation functions -----------

def compute_face_normals_and_areas(mesh):
    """
    Computes normal and area for each triangle face in the mesh.

    Returns:
        normals (np.ndarray): An (F, 3) array of unit normal vectors for each face.
        areas (np.ndarray): A (F,) array of areas for each face.

    Method:
        - Extract vertex indices for each triangle face.
        - Compute two edge vectors of the triangle.
        - Use cross product to get the face normal (not normalized).
        - Normalize to get unit normals.
        - Compute triangle area as half of the magnitude of the cross product.
    """
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    v0 = mesh.points[faces[:, 0]]
    v1 = mesh.points[faces[:, 1]]
    v2 = mesh.points[faces[:, 2]]
    
    cross = np.cross(v1 - v0, v2 - v0)
    areas = 0.5 * np.linalg.norm(cross, axis=1)
    normals = cross / np.maximum(np.linalg.norm(cross, axis=1, keepdims=True), 1e-8)
    return normals, areas

def build_vertex_to_faces_map(mesh):
    """
    Builds a mapping from each vertex index to the list of face indices that include that vertex.

    The same with pyvista's mesh.point_cell_ids()
    Returns:
        defaultdict(list): A dictionary where each key is a vertex index, and the value is 
        a list of face indices (from mesh.faces) that contain the vertex.

    Example:
        {
            0: {1, 5, 10},  # vertex 0 is part of faces 1, 5, and 10
            1: {0, 2},      # vertex 1 is part of faces 0 and 2
            ...
        }    
    """
    vertex_faces = defaultdict(list)
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    for i, (v1, v2, v3) in enumerate(faces):
        vertex_faces[v1].append(i)
        vertex_faces[v2].append(i)
        vertex_faces[v3].append(i)
    return vertex_faces

def compute_area_weighted_vertex_normals(mesh):
    """
    Computes unit vertex normals by averaging adjacent face normals, weighted by face area.

    Returns:
        vertex_normals (np.ndarray): An (N, 3) array of unit normal vectors for each vertex.

    Method:
        1. Compute face normals and face areas using cross products.
        2. Build a mapping from each vertex to the list of adjacent face indices.
        3. For each vertex:
            - Retrieve adjacent faces.
            - Average their normals, weighted by face area.
            - Normalize the result to obtain a unit normal.
    """
    face_normals, face_areas = compute_face_normals_and_areas(mesh)
    vertex_faces = build_vertex_to_faces_map(mesh)
    num_vertices = mesh.points.shape[0]
    vertex_normals = np.zeros((num_vertices, 3))

    for vi in range(num_vertices):
        adjacent_faces = vertex_faces[vi]
        if not adjacent_faces:
            continue
        areas = face_areas[adjacent_faces]
        normals = face_normals[adjacent_faces]
        weighted_sum = np.sum(normals * areas[:, np.newaxis], axis=0)
        vertex_normals[vi] = weighted_sum / np.linalg.norm(weighted_sum)
    
    return vertex_normals

# -------------------------- Normal vector angle --------------

def normal_vector_angle(n_p, n_q, degrees=False):
    """
    Parameters:
    - n_p, n_q: 3D vectors (can be non-normalized).
    - degrees: If True, return angle in degrees. Otherwise, radians.

    Returns:
    - Angle between vectors
    """
    dot_product = np.dot(n_p, n_q)
    norm_p = np.linalg.norm(n_p)
    norm_q = np.linalg.norm(n_q)

    if norm_p == 0 or norm_q == 0:
        return 0.0

    cos_theta = np.clip(dot_product / (norm_p * norm_q), -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    return np.degrees(angle_rad) if degrees else angle_rad

# ----------------------------------------------
def sorting_flat_vertices_with_flatness(normals, adjacency):
    """
    Find all flat vertices, compute their flatness (average normal angle to neighbors),
    and return them sorted by flatness (ascending).
    
    Returns:
        List of (vertex_index, flatness_value) tuples, sorted by flatness_value.
    """
    flat_vertices = []
    for vi, neighbors in adjacency.items():
        if not neighbors:
            continue
        # Check if all neighbor angles are below threshold
        angles = [normal_vector_angle(normals[vi], normals[vj]) for vj in neighbors]
        flatness = sum(angles) / len(angles)
        flat_vertices.append((vi, flatness))
        # if all(angle < threshold for angle in angles):
        #     flatness = sum(angles) / len(angles)
        #     flat_vertices.append((vi, flatness))
        
    # Sort by flatness value (ascending)
    flat_vertices.sort(key=lambda x: x[1])
    return flat_vertices

def cluster_surface(adj, v_normals, flat_v_sorted, weight, threshold):
    '''
    Parameters:
    adj: A dictionary where each key is a vertex index, and 
         the value is a set of its neighboring vertex indices.

    v_normals: An (N, 3) array of unit normal vectors for each vertex.

    flat_v_sorted: 1D list of vertex_index, sorted by flatness_value.

    weight: parameter for similarity function

    threshold: If similarity function < threshold: add vertex to a cluster
    '''
    flat_v_sorted_copy = copy.copy(flat_v_sorted)
    unvisited = set(range(len(v_normals)))
    clusters = []

    while unvisited:
        start = flat_v_sorted_copy.pop(0)
        if start not in unvisited:
            continue 
        cluster = set([start]) # Step 2 of the section 3's algorithm
        queue = deque([start])
        unvisited.remove(start)
        while queue:
            vi = queue.popleft()
            vi_neighbors = adj[vi]
            for vj in vi_neighbors:
                # Average angle btw
                avg_angle_vi = np.mean([normal_vector_angle(v_normals[vi], v_normals[vj_neighbor]) for vj_neighbor in adj[vj]])

                # Compute average normal vector all vertices in cluster by summing and normalizing
                normals_sum = np.sum([v_normals[vk] for vk in cluster], axis=0)
                normals_normalize = normals_sum / np.maximum(np.linalg.norm(normals_sum), 1e-8)
                cluster_normal = normals_normalize

                avg_angle_cluster = np.mean([normal_vector_angle(cluster_normal, v_normals[vj_neighbor]) for vj_neighbor in adj[vj]])
                # Proximity computation
                proximity = weight * avg_angle_vi + (1 - weight) * avg_angle_cluster

                if proximity < threshold:
                    if vj in unvisited:
                        cluster.add(vj)
                        queue.append(vj)
                        unvisited.remove(vj)

        clusters.append(cluster)
    return clusters

def merge_noise_faces(clusters, vertex_normals, adjacency, K=10):
    """
    Merge small (noise) clusters into neighboring real clusters based on normal similarity,
    while avoiding merging adjacent noise clusters together.

    Parameters:
        clusters (List[Set[int]]): List of vertex index sets (each cluster).
        vertex_normals (np.ndarray): (N, 3) array of vertex normals.
        adjacency (dict): {vertex_index: set(neighboring vertex indices)}.
        K (int): Max vertex count for a cluster to be considered noise.

    Returns:
        List[Set[int]]: Updated clusters with noise surfaces merged into real surfaces.
    """
    # Separate clusters
    real_clusters = [c for c in clusters if len(c) >= K]
    noise_clusters = [c for c in clusters if len(c) < K]

    # Precompute normals
    def cluster_avg_normal(cluster):
        normals = [vertex_normals[vi] for vi in cluster]
        summed = np.sum(normals, axis=0)
        return summed / np.linalg.norm(summed) if np.linalg.norm(summed) > 0 else summed

    real_normals = [cluster_avg_normal(c) for c in real_clusters]
    noise_normals = [cluster_avg_normal(c) for c in noise_clusters]

    # Create index-to-cluster maps for fast lookup
    vertex_to_real_cluster = {}
    for idx, cluster in enumerate(real_clusters):
        for v in cluster:
            vertex_to_real_cluster[v] = idx

    merged_indices = set()  # track merged noise cluster indices

    for ni, noise in enumerate(noise_clusters):
        if ni in merged_indices:
            continue
        noise_normal = noise_normals[ni]

        # Find adjacent real clusters only
        neighbor_real_candidates = set()
        for vi in noise:
            for vj in adjacency[vi]:
                if vj in vertex_to_real_cluster:
                    neighbor_real_candidates.add(vertex_to_real_cluster[vj])

        # Find the best matching real cluster by normal vector angle
        best_real_idx = None
        best_angle = float('inf')

        for r_idx in neighbor_real_candidates:
            angle = normal_vector_angle(noise_normal, real_normals[r_idx])
            if angle < best_angle:
                best_real_idx = r_idx
                best_angle = angle

        # Merge into best-matching real cluster
        if best_real_idx is not None:
            real_clusters[best_real_idx].update(noise)
            merged_indices.add(ni)
        # else: discard the noise cluster (could also be kept separately)

    return real_clusters


'''
    This code implement the Laplace smoothing algorithm mentioned in the section 2 of the paper 
    "Robust surface segmentation and edge feature lines extraction from fractured
    ragments of relics" 
'''


# ---------- Laplacian Smoothing Functions ----------

def build_vertex_adjacency(faces):
    """
    Builds a dictionary where each key is a vertex index, and the value is a set of its neighboring vertex indices.

    The same with pyvista's mesh.point_neighbors()
    """
    adjacency = defaultdict(set)
    for three_vertices in faces:
        i, j, k = three_vertices
        adjacency[i].update([j, k])
        adjacency[j].update([i, k])
        adjacency[k].update([i, j])
    return adjacency

def weighted_laplacian_smoothing(vertices, faces, iterations=1):
    vertices = vertices.copy()
    adjacency = build_vertex_adjacency(faces)

    for _ in range(iterations):
        new_vertices = vertices.copy()
        for i in range(len(vertices)):
            neighbors = list(adjacency[i])
            if not neighbors:
                continue

            weights = []
            for j in neighbors:
                dist = np.linalg.norm(vertices[i] - vertices[j])
                weight = 1.0 / (dist + 1e-8)  # Avoid division by zero
                weights.append(weight)
            weights = np.array(weights)
            weights /= np.sum(weights)  # Normalize λ_ij

            weighted_sum = sum(weights[k] * (vertices[neighbors[k]] - vertices[i]) for k in range(len(neighbors)))
            new_vertices[i] = vertices[i] + weighted_sum

        vertices = new_vertices

    return vertices

def compute_mesh_smoothness(vertices, faces): # To see how much a mesh is smooth
    adjacency = build_vertex_adjacency(faces)
    smoothness_vals = []

    for i in range(len(vertices)):
        neighbors = list(adjacency[i])
        if not neighbors:
            continue
        avg_neighbor = np.mean(vertices[neighbors], axis=0)
        smoothness = np.linalg.norm(vertices[i] - avg_neighbor)
        smoothness_vals.append(smoothness)

    return np.mean(smoothness_vals)

In [2]:
import numpy as np

def convert_to_builtin_int(obj):
    if isinstance(obj, dict):
        return {convert_to_builtin_int(k): convert_to_builtin_int(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_builtin_int(i) for i in obj]
    elif isinstance(obj, set):
        return {convert_to_builtin_int(i) for i in obj}
    elif isinstance(obj, tuple):
        return tuple(convert_to_builtin_int(i) for i in obj)
    elif isinstance(obj, np.integer):  # Catch np.int64, np.int32, ...
        return int(obj)
    else:
        return obj


In [3]:
import os
import pyvista as pv

mesh = pv.read('D:/Learn_and_Study/USTH/Bachelor/3D_Project/CG_dataset/brick_part01.obj')
# mesh = pv.examples.download_bunny()

In [4]:
faces_1D = mesh.faces
faces_nD = faces_1D.reshape((-1, 4))[:, 1:]

smoothed_vertices = weighted_laplacian_smoothing(mesh.points, faces_nD, 3)
smoothed_mesh = pv.PolyData(smoothed_vertices, faces_1D)


In [5]:
vertices = smoothed_mesh.points 
faces_1D = smoothed_mesh.faces
faces_nD = smoothed_mesh.faces.reshape((-1, 4))[:, 1:]

In [6]:
adj = build_vertex_adjacency(faces_nD)
adj = convert_to_builtin_int(adj)

v_normals = compute_area_weighted_vertex_normals(smoothed_mesh)

flat_v_sorted = sorting_flat_vertices_with_flatness(v_normals, adj)
flat_v_sorted = [vertex for vertex, flatness in flat_v_sorted]
flat_v_sorted = convert_to_builtin_int(flat_v_sorted)

### Test cluster_surface()

In [7]:
clusters = cluster_surface(adj, v_normals, flat_v_sorted, 0.5, 0.5)

### Test merge_noise_face() to remove noise faces

In [8]:
merged = merge_noise_faces(clusters, v_normals, adj, K=10)

In [9]:
colors = [
    'red', 'green', 'blue', 'yellow', 'magenta', 'cyan',
    'orange', 'lime', 'deeppink', 'deepskyblue', 'gold',
    'indigo', 'teal', 'crimson', 'mediumvioletred',
    'chartreuse', 'orangered', 'darkturquoise',
    'slateblue', 'darkgoldenrod'
]

plotter = pv.Plotter(shape=(1, 3))

plotter.subplot(0, 0)
plotter.add_title("Original")
plotter.add_mesh(smoothed_mesh, show_edges=True)

plotter.subplot(0, 1)
plotter.add_title("Clusters")
plotter.add_mesh(smoothed_mesh, show_edges=True)
for i, cluster in enumerate(clusters):
    cluster_indices = list(cluster)
    color = colors[i % len(colors)]
    plotter.add_mesh(smoothed_mesh.points[cluster_indices], color=color, point_size=10)


plotter.subplot(0, 2)
plotter.add_title("Merged")
plotter.add_mesh(smoothed_mesh, show_edges=True)
for i, cluster in enumerate(merged):
    merged_indices = list(cluster)
    color = colors[i % len(colors)]
    plotter.add_mesh(smoothed_mesh.points[merged_indices], color=color, point_size=10)

plotter.link_views()
plotter.show()


Widget(value='<iframe src="http://localhost:50797/index.html?ui=P_0x189b98c4680_0&reconnect=auto" class="pyvis…

### Test surface differentiation

In [10]:
def compute_local_bending_energy(vertices, vertex_normals, adjacency):
    """
    Computes local bending energy e_k(p) for each vertex p as defined in the paper.

    Parameters:
        vertices (np.ndarray): (N, 3) array of vertex positions.
        vertex_normals (np.ndarray): (N, 3) array of vertex normals.
        adjacency (dict): {vertex_index: set of neighbor vertex indices}

    Returns:
        e_k (np.ndarray): (N,) array of local bending energy per vertex.
    """
    N = len(vertices)
    e_k = np.zeros(N)

    for p in range(N):
        neighbors = adjacency[p]
        if not neighbors:
            e_k[p] = 0
            continue

        total = 0.0
        for q in neighbors:
            norm_diff = np.linalg.norm(vertex_normals[p] - vertex_normals[q]) ** 2
            geom_dist = np.linalg.norm(vertices[p] - vertices[q]) ** 2 + 1e-8  # avoid div by zero
            total += norm_diff / geom_dist

        e_k[p] = total / len(neighbors)

    return e_k

from scipy.spatial import cKDTree
import numpy as np

def compute_integrated_bending_energy_fast(e_k, vertices, radius):
    """
    Fast computation of e_{k,r}(p) using cKDTree for efficient radius search.
    """
    N = len(vertices)
    e_kr = np.zeros(N)

    # Build KDTree for fast radius neighbor queries
    tree = cKDTree(vertices)

    # Query all neighbors within radius r
    all_neighbors = tree.query_ball_point(vertices, r=radius)

    for i in range(N):
        neighbors = all_neighbors[i]
        if not neighbors:
            e_kr[i] = e_k[i]  # fallback if no neighbors
        else:
            e_kr[i] = np.mean(e_k[neighbors])
    print(all_neighbors)
    return e_kr

# def classify_faces_by_roughness(mesh, e_kr, threshold=0.05):
    

In [ ]:
e_k = compute_local_bending_energy(vertices, v_normals, adj)
e_kr = compute_integrated_bending_energy_fast(e_k, vertices, 10)

In [ ]:
# Get the index of the face color

# Fractured surface
magenta_index = colors.index('magenta')  # 4
magenta_points = list(clusters[magenta_index]) if magenta_index < len(clusters) else []

cyan_index = colors.index('cyan')  # 4
cyan_points = list(clusters[cyan_index]) if cyan_index < len(clusters) else []

# Original surface
red_index = colors.index('red')  # 4
red_points = list(clusters[red_index]) if red_index < len(clusters) else []

blue_index = colors.index('blue')  # 4
blue_points = list(clusters[blue_index]) if blue_index < len(clusters) else []



In [ ]:
# print(len(magenta_points))
# print(len(cyan_points))
# print(len(red_points))
# print(len(blue_points))

In [ ]:
# original_indices = red_points[:50] + blue_points[:50]
# fractured_indices = cyan_points[:50] + magenta_points[:50]


In [ ]:
original_indices = merged[0][:50] + merged[2][:50]
fractured_indices = merged[5][:50] + merged[4][:50]

In [ ]:
original_values = e_kr[original_indices]
fractured_values = e_kr[fractured_indices]

import matplotlib.pyplot as plt

plt.hist(original_values, bins=30, alpha=0.6, label='Original')
plt.hist(fractured_values, bins=30, alpha=0.6, label='Fractured')
plt.axvline(x=np.mean(original_values), color='blue', linestyle='--')
plt.axvline(x=np.mean(fractured_values), color='red', linestyle='--')
plt.legend()
plt.title("Distribution of e_{k,r}(p)")
plt.xlabel("Integrated Bending Energy")
plt.ylabel("Frequency")
plt.show()



In [ ]:
fractured_values

In [ ]:
from scipy.stats import norm

mu1, std1 = np.mean(original_values), np.std(original_values)
mu2, std2 = np.mean(fractured_values), np.std(fractured_values)

# Solve for x in norm1(x) == norm2(x)
a = 1/(2*std1**2) - 1/(2*std2**2)
b = mu2/(std2**2) - mu1/(std1**2)
c = (mu1**2)/(2*std1**2) - (mu2**2)/(2*std2**2) - np.log(std2/std1)

discriminant = b**2 - 4*a*c
x1 = (-b + np.sqrt(discriminant)) / (2*a)
x2 = (-b - np.sqrt(discriminant)) / (2*a)

# Choose the threshold between the two means
threshold = x1 if mu1 < x1 < mu2 or mu2 < x1 < mu1 else x2
threshold

In [ ]:
# import nbformat

# # 📌 BƯỚC 1: Đặt tên file notebook và file xuất ra
# notebook_file = "pyvista_learning.ipynb"     # 👉 đổi thành tên notebook của bạn
# output_file = "extract_code_from_ipynb.py"      # 👉 tên file .py để lưu code

# # 📌 BƯỚC 2: Đọc file notebook
# with open(notebook_file, "r", encoding="utf-8") as f:
#     nb = nbformat.read(f, as_version=4)

# # 📌 BƯỚC 3: Giả sử ô hiện tại đang chạy là ô cuối cùng (có thể điều chỉnh nếu cần)
# current_cell_index = len(nb.cells) - 1

# # 📌 BƯỚC 4: Lấy code từ tất cả các cell code phía trên
# code_above = ""
# for i in range(current_cell_index):
#     cell = nb.cells[i]
#     if cell.cell_type == "code":
#         code_above += cell.source + "\n\n"

# # 📌 BƯỚC 5: Lưu vào file .py
# with open(output_file, "w", encoding="utf-8") as f:
#     f.write(code_above)

# print(f"✅ Đã lưu mã từ các cell phía trên vào file: {output_file}")
